In [1]:
# Mudando para a pasta raíz
import sys
sys.path.append("..")

from src.io_operations import read_yaml, read_tiff, save_yaml, fix_relative_paths, get_image_metadata, array2raster
from src.utils import check_folder
from src.metrics import evaluate_metrics, evaluate_component_metrics
from sample_selection import get_components_stats, filter_components_by_mask

from utils import *
import os
from os.path import dirname, join

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from skimage.measure import label, find_contours
from skimage.color import label2rgb, color_dict
from skimage.transform import resize
from sklearn.metrics import jaccard_score

from tqdm import tqdm

from millify import millify
sns.set_style("whitegrid")
import torchvision.transforms.functional as F
from scipy.ndimage import gaussian_filter
import gc
import torch

In [8]:
DOWNSAMPLE_FACTOR = 4

In [9]:
OUTPUT_FOLDER = f"../{DOWNSAMPLE_FACTOR}x_amazon_input_data"
check_folder(OUTPUT_FOLDER)

In [10]:
check_folder(join(OUTPUT_FOLDER, "segmentation"))
check_folder(join(OUTPUT_FOLDER, "orthoimage"))

In [45]:
orthoimage_path = r"/u00/luiz.luz/multi-task-fcn/amazon_input_data/orthoimage/NOV_2017_FINAL_004.tif"
orthoimage = read_tiff(orthoimage_path)

orthoimage_metadata = get_image_metadata(orthoimage_path)

In [46]:
test_path = "../amazon_input_data/segmentation/test_set.tif"
test_set = read_tiff(
    test_path
)
test_set_metadata = get_image_metadata(test_path)

In [47]:
train_path = "../amazon_input_data/segmentation/train_set.tif"
train_set = read_tiff(
    train_path
)
train_set_metadata = get_image_metadata(train_path)

In [11]:
mask_path = "../amazon_input_data/mask.tif"
mask = read_tiff(
    mask_path
)
mask_metadata = get_image_metadata(mask_path)

In [49]:
resized_orthoimage = resize(orthoimage, (orthoimage.shape[0], orthoimage.shape[1]//DOWNSAMPLE_FACTOR, orthoimage.shape[2]//DOWNSAMPLE_FACTOR), anti_aliasing=True)
resized_orthoimage = (resized_orthoimage*255).astype(np.uint8)

In [50]:
resized_test_set = resize(test_set, (test_set.shape[0]//DOWNSAMPLE_FACTOR, test_set.shape[1]//DOWNSAMPLE_FACTOR), anti_aliasing=False, order=0)

In [51]:
resized_train_set = resize(train_set, (train_set.shape[0]//DOWNSAMPLE_FACTOR, train_set.shape[1]//DOWNSAMPLE_FACTOR), anti_aliasing=False, order=0)

In [12]:
resized_mask = resize(mask, (mask.shape[0]//DOWNSAMPLE_FACTOR, mask.shape[1]//DOWNSAMPLE_FACTOR), anti_aliasing=False, order=0)

In [53]:
array2raster(
    path_to_save=join(OUTPUT_FOLDER,"orthoimage", "orthoimage.tif"),
    array=resized_orthoimage,
    image_metadata=orthoimage_metadata,
    dtype="uint8"
)

In [54]:
array2raster(
    path_to_save=join(OUTPUT_FOLDER, "segmentation", "test_set.tif"),
    array=resized_test_set,
    image_metadata=test_set_metadata,
    dtype="uint8"
)

In [55]:
array2raster(
    path_to_save=join(OUTPUT_FOLDER, "segmentation", "train_set.tif"),
    array=resized_train_set,
    image_metadata=train_set_metadata,
    dtype="uint8"
)

In [13]:
array2raster(
    path_to_save=join(OUTPUT_FOLDER, "mask.tif"),
    array=resized_mask,
    image_metadata=mask_metadata,
    dtype="uint8"
)